In [1]:
from math import ceil
from spatial_scan import scan
from vae import VAE, encoder, decoder, tf, keras

batch_len = 1000
    
with open("../test_30_1_0.3_100000_0.1_x.txt") as f:
    data = [[float(i.strip()) for i in s.split(" ") if i!=""] for s in f.read().split("\n") if s!=""]
    xyz=[[[data[h][i+3*j]for i in range(3)]for j in range(len(data[h])//3)]for h in range(len(data))]
    del data

# print(len(data), [len(data[i]) for i in range(len(data))])
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint("./checkpoints", save_best_only=True)
    
if False:#len(argv)>2 and argv[2]=="--resume":
    vae=keras.models.load_model("./model_info/")
else:
    vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())

2023-05-12 11:14:48.612114: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-12 11:14:51.392220: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-12 11:14:51.392425: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-12 11:14:51.392450: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [7]:
for i in range(ceil(len(xyz)/batch_len)):
    bdata = [] # will hold bucketed data version
    for j in range(batch_len*i, batch_len*(i+1)):
        if j>len(xyz):
            break
        #print(xyz[j])
        bdata.append(scan(xyz[j],4,4,4))
    dataset = tf.data.Dataset.from_tensor_slices((bdata,))
    vae.fit(dataset, epochs=30, batch_size=128, callbacks=[model_checkpoint_callback])

vae.load_weights("./checkpoints")
vae.save("./model_info/")
    

Epoch 1/30


ValueError: in user code:

    File "/home/davide/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/davide/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/davide/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/davide/script/uni/variational-autoencoders/vae/vae.py", line 71, in train_step
        z_mean, z_log_var, z = self.encoder(data)
    File "/home/davide/.local/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer 'max_pooling3d' (type MaxPooling3D).
    
    Negative dimension size caused by subtracting